# 电商用户行为分析

## 项目背景及分析目标：数据是某电商平台某段时期内的用户行为数据，以用户增长为目标，通过分析用户行为和商品购买数据来总结问题，为用户运营提供建议。

## 整体分析思路：我们整体分析方案是从整体指标，用户行为分析以及商品分析三个方面来进行，整体建立相关监测指标来观察平台用户的质量和行为变化，接下来我们按照用户增长中的RARRA模型的各个环节来做用户行为分析，最后我们从商品的角度来分析热卖商品，并对用户做了基于物品协同过滤的商品推荐。

## 数据说明：
- user_id：用户身份，脱敏
- item_id：商品ID，脱敏
- behavior_type：用户行为类型（包含点击、收藏、加购物车、支付四种行为，分别用数字1、2、3、4表示）
- user_geohash：地理位置
- item_category：品类ID（商品所属的品类）
- time：用户行为发生的时间

## 数据描述和数据清洗

In [1]:
import numpy as np
import pandas as pd
from pyecharts.charts import *
import pyecharts.options as opts
from pyecharts.components import Table
from pyecharts.globals import ThemeType
from pyecharts.commons.utils import JsCode
from pyecharts.options import ComponentTitleOpts
import warnings
from sklearn.cluster import KMeans
from datetime import datetime
from datetime import timedelta
#忽略警告
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('F:/迅雷下载/数据集/淘宝用户行为.csv')
data.head()

,user_id,item_id,behavior_type,user_geohash,item_category,time
0,98047837,232431562,1,NaN,4245,2014-12-06 02
1,97726136,383583590,1,NaN,5894,2014-12-09 20
2,98607707,64749712,1,NaN,2883,2014-12-18 11
3,98662432,320593836,1,96nn52n,6562,2014-12-06 10
4,98145908,290208520,1,NaN,13926,2014-12-16 21


查看数据基本信息

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12256906 entries, 0 to 12256905
Data columns (total 6 columns):
user_id          int64
item_id          int64
behavior_type    int64
user_geohash     object
item_category    int64
time             object
dtypes: int64(4), object(2)
memory usage: 561.1+ MB


In [4]:
data.describe()

,user_id,item_id,behavior_type,item_category
count,1.225691e+07,1.225691e+07,1.225691e+07,1.225691e+07
mean,7.170732e+07,2.023084e+08,1.105271e+00,6.846162e+03
std,4.122920e+07,1.167397e+08,4.572662e-01,3.809922e+03
min,4.913000e+03,6.400000e+01,1.000000e+00,2.000000e+00
25%,3.584965e+07,1.014130e+08,1.000000e+00,3.721000e+03
50%,7.292804e+07,2.021359e+08,1.000000e+00,6.209000e+03
75%,1.073774e+08,3.035405e+08,1.000000e+00,1.029000e+04
max,1.424559e+08,4.045625e+08,4.000000e+00,1.408000e+04


In [5]:
data.describe(include=['O'])

,user_geohash,time
count,3922082,12256906
unique,575458,744
top,94ek6ke,2014-12-11 22
freq,1052,54797


查看缺失值

In [6]:
data.isnull().sum() 

user_id                0
item_id                0
behavior_type          0
user_geohash     8334824
item_category          0
time                   0
dtype: int64

发现user_geohash列缺失严重，而该列对分析没有帮助，删去

In [7]:
del data['user_geohash']

将时间拆分为日期，小时

In [8]:
data['date'] = data['time'].apply(lambda x:x.split(' ')[0])
data['hour'] = data['time'].apply(lambda x:x.split(' ')[1])

In [9]:
data['time']=pd.to_datetime(data['time'])
data['date']=pd.to_datetime(data['date'])
data['hour']=data['hour'].astype('int64')

将behavior_type调整为对应的行为，{1:pv,	2:fav,	3:cart	,4:buy}

In [10]:
data.loc[data['behavior_type']==1,'behavior_type']='pv'
data.loc[data['behavior_type']==2,'behavior_type']='fav'
data.loc[data['behavior_type']==3,'behavior_type']='cart'
data.loc[data['behavior_type']==4,'behavior_type']='buy'

In [11]:
data.head()

,user_id,item_id,behavior_type,item_category,time,date,hour
0,98047837,232431562,pv,4245,2014-12-06 02:00:00,2014-12-06,2
1,97726136,383583590,pv,5894,2014-12-09 20:00:00,2014-12-09,20
2,98607707,64749712,pv,2883,2014-12-18 11:00:00,2014-12-18,11
3,98662432,320593836,pv,6562,2014-12-06 10:00:00,2014-12-06,10
4,98145908,290208520,pv,13926,2014-12-16 21:00:00,2014-12-16,21


## 整体指标：日新增用户、PV、UV、MAU和DAU、用户留存率、交易用户比例、复购率、跳失率。

日新增用户：由于本数据集中没有用户的注册时间，所以把用户在平台的行为日期作为注册时间。

In [12]:
data_user_firstday=data.groupby('user_id')['date'].min().reset_index()
data_new_users_count=data_user_firstday.groupby('date')['user_id'].nunique().reset_index()

In [13]:
data_new_users_count.head()

,date,user_id
0,2014-11-18,6343
1,2014-11-19,1283
2,2014-11-20,550
3,2014-11-21,340
4,2014-11-22,250


In [14]:
line = (Line()
        .add_xaxis(data_new_users_count['date'].astype(str))
        .add_yaxis('new_user_counts', data_new_users_count['user_id'].tolist())
        .set_global_opts(
        tooltip_opts=opts.TooltipOpts(
            is_show=True,trigger="axis",axis_pointer_type="cross"
        ),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            axispointer_opts=opts.AxisPointerOpts(is_show=True,type_="shadow"),
        ),
        yaxis_opts=opts.AxisOpts(
            name="日新增用户数量",
            type_="value",
            axislabel_opts=opts.LabelOpts(formatter="{value}"),
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        title_opts=opts.TitleOpts(title="日新增用户数量图",pos_left='center'),
        legend_opts=opts.LegendOpts(is_show=True,pos_top='95%')
    ) 
        .set_series_opts(
        # 为了不影响标记点，这里把标签关掉
        label_opts=opts.LabelOpts(is_show=False)
        ))
      

line.render_notebook()

新用户数量在第一天最多达6300多人，后面快速回落到几十人的水平，这是因为我们的数据选用的注册日是用户第一天登录的日期的原因。
跳失率：

In [15]:
user_type=data.groupby('user_id')['behavior_type'].value_counts().unstack()

user_type_only_pv_num=user_type[user_type['pv']==user_type.sum(axis=1)]['pv'].count()
only_ratio=user_type_only_pv_num/user_type['pv'].count()*100
user_type.head()
print(only_ratio)

3.4099999999999997


In [16]:
l1 = (
    Liquid()
    .add("跳失率", [0.034, 0.04], center=["60%", "50%"])
    .set_global_opts(title_opts=opts.TitleOpts(title="用户跳失率"),
                    legend_opts=opts.LegendOpts(is_show=True)
    )
)
l1.render_notebook()

其他指标会在下文中详细给出。从这些整体指标中我们可以看到，平台付费率很高达到了88.86%,跳失率很低只有3%。我们只需要做的是提高用户忠诚度。

## 用户行为分析:
1. 用户留存:
    - a)	按日维度和周维度分析用户留存率。

日维度留存计算，由于项目中没有给注册时间，以用户首次访问为注册时间

In [17]:


def retention(df,n):
    n_date = pd.Series(df.date.unique()).sort_values()[:-n] 
           
    retention_rates = []
    user = []
    for i in n_date:
        new_user = set(df[df.date == i]['user_id'].unique()) - set(user)
        # set() 函数创建一个无序不重复元素集，可进行关系测试，删除重复数据，还可以计算交集、差集、并集等。
        user.extend(new_user)
        n_new_user = df[df.date == i+timedelta(n)]['user_id'].unique()
        a = 0
        for j in n_new_user:
            if j in new_user:
                a+=1
        #print(a,len(new_user))
        retention_rate = a/len(new_user)
        retention_rates.append(retention_rate)
        #print(len(retention_rates),len(n_date))
    total_retention_rate = pd.DataFrame({'date':n_date,'留存率':retention_rates})
    return total_retention_rate

retention_day_1=retention(data,1)
retention_day_3=retention(data,3)
retention_day_5=retention(data,5)
retention_day_7=retention(data,7)

In [18]:
attr2 = retention_day_1.date.astype(str).tolist()
line = (Line()
        .add_xaxis(attr2)
        .add_yaxis('7日留存率', retention_day_7.留存率.tolist())
        .add_yaxis('5日留存率', retention_day_5.留存率.tolist())
        .add_yaxis('3日留存率', retention_day_3.留存率.tolist())
        .add_yaxis('1日留存率', retention_day_1.留存率.tolist())
        .set_global_opts(
        tooltip_opts=opts.TooltipOpts(
            is_show=True,trigger="axis",axis_pointer_type="cross"
        ),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            axispointer_opts=opts.AxisPointerOpts(is_show=False,type_="shadow"),
        ),
        yaxis_opts=opts.AxisOpts(
            name="留存率",
            type_="value",
            min_=0,
            max_=1,
            interval=0.2,
            axislabel_opts=opts.LabelOpts(formatter="{value}"),
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        title_opts=opts.TitleOpts(title="日留存率对比分析图",pos_left='center'),
        legend_opts=opts.LegendOpts(is_show=True,pos_top='95%')
    )
        .set_series_opts(
        # 为了不影响标记点，这里把标签关掉
        label_opts=opts.LabelOpts(is_show=False))
      )

line.render_notebook()

周维度留存计算

In [19]:
def retention(df,n):
    n_date = pd.Series(df.date.unique()).sort_values()[:-(n*7+len(df)-len(df)//7*7)].reset_index(drop=True)    
    retention_rates = []
    user = []
    datalist=[]
    for i in range(0,len(n_date),7):
        datalist.append('第{:.0f}周'.format(i/7+1))
        new_user = set(df[(df.date >= n_date[i])&(df.date <= (n_date[i]+timedelta(6)))]['user_id'].unique()) - set(user)
        # set() 函数创建一个无序不重复元素集，可进行关系测试，删除重复数据，还可以计算交集、差集、并集等。
        user.extend(new_user)
        n_new_user = df[(df.date >= (n_date[i]+timedelta(7)))&(df.date <= (n_date[i]+timedelta(13)))]['user_id'].unique()
        a = 0
        for j in n_new_user:
            if j in new_user:
                a+=1
        #print(a,len(new_user))
        retention_rate = a/len(new_user)
        retention_rates.append(retention_rate)
        #print(len(retention_rates),len(n_date))
    total_retention_rate = pd.DataFrame({'date':datalist,'留存率':retention_rates})
    return total_retention_rate

retention_day_1=retention(data,1)
retention_day_2=retention(data,2)
retention_day_3=retention(data,3)

In [20]:
attr=retention_day_1.date.astype(str).tolist()
line=(Line().
      add_xaxis(attr)
      .add_yaxis('第一周留存率',retention_day_1['留存率'].tolist())
      .add_yaxis('第二周留存率',retention_day_2['留存率'].tolist())
      .add_yaxis('第三周留存率',retention_day_3['留存率'].tolist())
      .set_global_opts(
      tooltip_opts=opts.TooltipOpts(is_show=True,trigger='axis',
                                   axis_pointer_type='cross'
                                   ),
          xaxis_opts=opts.AxisOpts(
         type_='category',
              axispointer_opts=opts.AxisPointerOpts(is_show=True,type_='shadow')
          ),
          yaxis_opts=opts.AxisOpts(name='留存率',type_='value',
                                  min_=0,max_=1,interval=0.2,
                                   axislabel_opts=opts.LabelOpts(formatter="{value}"),
           axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
                                  ),
          title_opts=opts.TitleOpts(title='周留存率对比分析图',pos_left='center'),
          legend_opts=opts.LegendOpts(is_show=True,pos_top='95%')
      ).set_series_opts(
      label_opts=opts.LabelOpts(is_show=False)
      ))
line.render_notebook()

由于没有用户注册时间，所以我们把用户第一次的行为时间作为注册时间来计算留存率，导致计算的数据不是很准确，这里的日留存率看不出太明显的规律。

 - b)按渠道分析留存率，找到那些留存率低的获客渠道。 

2. 用户激活
     - a)分析DAU、MAU（如有较大变化拆分用户进行分析）

某段时间内有收藏、加购、或者购买行为的定义为活跃用户。

In [21]:
dau=data.groupby(['date','user_id']).agg({"behavior_type":lambda x:(x!='pv').sum()}).reset_index().sort_values(by=['date','user_id'])

In [22]:
dau=dau.rename(columns={'behavior_type':'活跃值'})

In [23]:
dau=dau.loc[dau['活跃值']>=0].groupby('date')['user_id'].nunique().reset_index().rename(columns={'user_id':'dau'})

In [24]:
dau.head()

,date,dau
0,2014-11-18,6343
1,2014-11-19,6420
2,2014-11-20,6333
3,2014-11-21,6276
4,2014-11-22,6187


In [25]:
line = (Line()
        .add_xaxis(dau['date'].astype(str))
        .add_yaxis('dau', dau['dau'].tolist())
        .set_global_opts(
        tooltip_opts=opts.TooltipOpts(
            is_show=True,trigger="axis",axis_pointer_type="cross"
        ),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            axispointer_opts=opts.AxisPointerOpts(is_show=True,type_="shadow"),
        ),
        yaxis_opts=opts.AxisOpts(
            name="DAU",
            type_="value",
            axislabel_opts=opts.LabelOpts(formatter="{value}"),
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        title_opts=opts.TitleOpts(title="DAU图",pos_left='center'),
        legend_opts=opts.LegendOpts(is_show=True,pos_top='95%')
    ) 
        .set_series_opts(
        # 为了不影响标记点，这里把标签关掉
        label_opts=opts.LabelOpts(is_show=False)
        ))
      

line.render_notebook()

从dau表中可以看到，日活用户在双十二活动前后有一个波峰值，而在其他日期比较平稳，可见活动取得了不错的效果。

In [26]:
mau=data.groupby('user_id').agg({"behavior_type":lambda x:(x!='pv').sum()}).reset_index().rename(columns={'behavior_type':'活跃值'})

In [27]:
mau_value=mau.loc[mau['活跃值']>0]['user_id'].nunique()
print('月活跃度为:{}'.format(mau_value))

月活跃度为:9659


 - b)分析小时维度上的PV，UV变化规律。

In [28]:
pv_hour=data.groupby('hour')['user_id'].count().reset_index().rename(columns={'user_id':'pv'})
uv_hour=data.groupby('hour')['user_id'].nunique().reset_index().rename(columns={'user_id':'uv'})
uv_hour

,hour,uv
0,0,5786
1,1,3780
2,2,2532
3,3,1937
4,4,1765
5,5,2030
6,6,3564
7,7,5722
8,8,7108
9,9,7734


In [29]:
line = (Line()
        .add_xaxis(pv_hour['hour'])
        .add_yaxis('页面访问量(pv)', pv_hour['pv']/10000)
        .add_yaxis('独立访客数(uv)',uv_hour['uv']
                  ,yaxis_index=1)
         .extend_axis(
        yaxis=opts.AxisOpts(
            name="uv",
            type_="value",
            min_=0,
            max_=10000,
            interval=1000,
            axislabel_opts=opts.LabelOpts(formatter="{value}人"),
        ))
        .set_global_opts(
        tooltip_opts=opts.TooltipOpts(
            is_show=True,trigger="axis",axis_pointer_type="cross"
        ),
        xaxis_opts=opts.AxisOpts(
            type_="value",
            axispointer_opts=opts.AxisPointerOpts(is_show=True,type_="shadow"),
        ),
        yaxis_opts=opts.AxisOpts(
            name="pv",
            type_="value",
            min_=0,
            max_=120,
            interval=20,
            axislabel_opts=opts.LabelOpts(formatter="{value} 万人"
        )),
        title_opts=opts.TitleOpts(title="小时维度下的PV和UV",pos_left='center'),
        legend_opts=opts.LegendOpts(is_show=True,pos_top='95%')
    ) 
        .set_series_opts(
        # 为了不影响标记点，这里把标签关掉
        label_opts=opts.LabelOpts(is_show=False)
        ))
      

line.render_notebook()

从pv小时图可以看到，用户数量在18-22点时pv有明显的升高趋势，在10点以后开始回落。所以在18-22点我们可以多在平台做运营推广的相关互动。

   - c)结合用户在用户引导环节中的数据埋点，总结忠诚用户在引导环节中的行为规律。在用户注册后，引导用户到前者的行为路径上来。 

 3. 用户自传播
  - a)通过检测内部用户做平台推广的行为数据，按推广活动划分，分析不同活动的效果
  - b)收集外部数据，分析用户对平台的反馈信息

4. 用户付费
  - a)	统计交易用户比例，分析用户各环节转化率漏斗，找到转化率流失严重的环节。

统计交易用户比例

In [30]:
all_user_count=data['user_id'].nunique()

In [31]:
all_user_count

10000

In [32]:
buy_user_count=data.loc[data['behavior_type']=='buy']['user_id'].nunique()
buy_user_count

8886

In [33]:
buy_ratio=buy_user_count/all_user_count*100
print('交易用户比例为{}%'.format(buy_ratio))

交易用户比例为88.86%


In [34]:
pv_df = data[data['behavior_type']=='pv']
buy_df = data[data['behavior_type']=='buy']
fav_cart_df = data[(data['behavior_type']=='cart')|(data['behavior_type']=='fav')]


In [35]:
pv_fav_cart=pd.merge(left=pv_df,right=fav_cart_df,how='inner',on=['user_id','item_id','item_category'],suffixes=['_pv','_cart_fav'])
fav_cart_buy_df=pd.merge(left=fav_cart_df,right=buy_df,how='inner',on=['user_id','item_id','item_category'],suffixes=['_cart_fav','buy'])

In [36]:
pv_num=pv_df.drop_duplicates(subset=['user_id','item_id','item_category'])['user_id'].count()
pv_fav_cart_num=pv_fav_cart.drop_duplicates(subset=['user_id','item_id','item_category'])['user_id'].count()
fav_cart_buy_num=fav_cart_buy_df.drop_duplicates(subset=['user_id','item_id','item_category'])['user_id'].count()
print(pv_num,pv_fav_cart_num,fav_cart_buy_num)

4677501 486434 66506


In [37]:
attr = ['点击', '加入购物车 or 收藏', '购买']

values = [np.around((pv_num / pv_num * 100), 2),
         np.around((pv_fav_cart_num/pv_num * 100), 2),
         np.around((fav_cart_buy_num / pv_num * 100), 2)]
values

[100.0, 10.4, 1.42]

In [38]:
funnel = (
    Funnel()
    .add("商品", [list(z) for z in zip(attr, values)])
    .set_global_opts(title_opts=opts.TitleOpts(title="用户购买行为整体转化漏斗"))
)

funnel.render_notebook()

由于数据中没有付费金额，这里我们定义ARPPU=平均每付费用户交易次数

In [39]:
arppu=data.loc[data['behavior_type']=='buy'].groupby('date').apply(lambda x:x['user_id'].count()/x['user_id'].nunique()).reset_index()

In [40]:
arppu.head()

,date,0
0,2014-11-18,2.423652
1,2014-11-19,2.439444
2,2014-11-20,2.320375
3,2014-11-21,2.271429
4,2014-11-22,2.530120


In [41]:
line = (Line()
        .add_xaxis(arppu['date'].astype(str))
        .add_yaxis('arppu', arppu.iloc[:,1])
        .set_global_opts(
        tooltip_opts=opts.TooltipOpts(
            is_show=True,trigger="axis",axis_pointer_type="cross"
        ),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            axispointer_opts=opts.AxisPointerOpts(is_show=True,type_="shadow"),
        ),
        yaxis_opts=opts.AxisOpts(
            name="ARPPU",
            type_="value",
            axislabel_opts=opts.LabelOpts(formatter="{value}"),
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        title_opts=opts.TitleOpts(title="ARPPU图",pos_left='center'),
        legend_opts=opts.LegendOpts(is_show=True,pos_top='95%')
    ) 
        .set_series_opts(
        # 为了不影响标记点，这里把标签关掉
        label_opts=opts.LabelOpts(is_show=False)
        ))
      

line.render_notebook()

付费转化周期

In [42]:
pv_time=data[data['behavior_type']=='pv'].groupby(['user_id','item_category','item_id'])['time'].agg({'time_min':lambda x:x.min()}).reset_index()

In [43]:
pv_time.head()
pv_time.size

18710004

In [44]:
buy_time=data[data['behavior_type']=='buy'].groupby(['user_id','item_category','item_id'])['time'].agg({'buy_time_min':lambda x:x.min()}).reset_index()

In [45]:
buy_time.head()

,user_id,item_category,item_id,buy_time_min
0,4913,5065,361346418,2014-12-16 12:00:00
1,4913,9127,44957435,2014-12-07 22:00:00
2,4913,11981,92674121,2014-12-13 18:00:00
3,4913,13381,172639700,2014-12-11 18:00:00
4,4913,13381,343170253,2014-12-01 23:00:00


In [46]:
pv_buy_time=pd.merge(left=pv_time,right=buy_time,how='inner',on=['user_id','item_category','item_id'])

In [47]:
pv_buy_time.head()


,user_id,item_category,item_id,time_min,buy_time_min
0,4913,5065,361346418,2014-12-13 21:00:00,2014-12-16 12:00:00
1,4913,9127,44957435,2014-12-07 21:00:00,2014-12-07 22:00:00
2,4913,11981,92674121,2014-12-13 17:00:00,2014-12-13 18:00:00
3,4913,13381,172639700,2014-12-11 17:00:00,2014-12-11 18:00:00
4,4913,13381,343170253,2014-11-26 18:00:00,2014-12-01 23:00:00


In [48]:
pv_buy_time['pv_buy_time_diff']=pv_buy_time.apply(lambda x:((x['buy_time_min']-x['time_min'])/pd.Timedelta(1, 'H')) if x['buy_time_min']>x['time_min'] else 0,axis=1)

In [49]:
pv_buy_time.head()

,user_id,item_category,item_id,time_min,buy_time_min,pv_buy_time_diff
0,4913,5065,361346418,2014-12-13 21:00:00,2014-12-16 12:00:00,63.0
1,4913,9127,44957435,2014-12-07 21:00:00,2014-12-07 22:00:00,1.0
2,4913,11981,92674121,2014-12-13 17:00:00,2014-12-13 18:00:00,1.0
3,4913,13381,172639700,2014-12-11 17:00:00,2014-12-11 18:00:00,1.0
4,4913,13381,343170253,2014-11-26 18:00:00,2014-12-01 23:00:00,125.0


In [50]:
pv_buy_time_mean=int(pv_buy_time.loc[pv_buy_time['buy_time_min']>pv_buy_time['time_min']]['pv_buy_time_diff'].mean())

In [51]:
print('付费平均周期为{}小时'.format(pv_buy_time_mean))

付费平均周期为58小时


重复消费用户比例

In [52]:
data_user_buytimes=data[data['behavior_type']=='buy'].groupby('user_id')['item_id'].count().reset_index()

In [53]:
data_user_buytimes.head()

,user_id,item_id
0,4913,6
1,6118,1
2,7528,6
3,7591,21
4,12645,8


In [54]:
user_buyTwoTimes_count=data_user_buytimes[data_user_buytimes.iloc[:,1]>=2]['item_id'].count()

In [55]:
user_buyTwoTimes_count

8148

In [56]:
user_buyTwoTimes_ratio=user_buyTwoTimes_count/data['user_id'].nunique()

In [57]:
print('重复购买的用户比例为{:.2f}%'.format(user_buyTwoTimes_ratio*100))

重复购买的用户比例为81.48%


从ARRPU图可以看到，用户在双12期间交易数量有明显的上升。平台重复交易用户比例为81.48%，说明平台用户多为重复交易用户，付费周期为58h。那么由此我们在58h内是提醒用户购买的黄金时段。

3. 用户价值分层(RFM)

In [58]:
date_max=data['date'].max()

In [59]:
buy_df=data[data['behavior_type']=='buy']

In [60]:
buy_df['diff']=buy_df['date'].apply(lambda x:(date_max-x).days)

In [61]:
rfm_df=buy_df.groupby('user_id').agg({'diff':'min','item_id':'count'}).reset_index()

In [62]:
rfm_df.head()

,user_id,diff,item_id
0,4913,2,6
1,6118,1,1
2,7528,5,6
3,7591,5,21
4,12645,4,8


In [63]:
rfm_df.describe().T

,count,mean,std,min,25%,50%,75%,max
user_id,8886.0,7.152087e+07,4.120719e+07,4913.0,35677312.75,72388002.0,107194492.5,142455899.0
diff,8886.0,5.811839e+00,6.678478e+00,0.0,1.00,4.0,7.0,30.0
item_id,8886.0,1.352746e+01,1.969879e+01,1.0,4.00,8.0,17.0,809.0


In [64]:
r_bins=[-0.1,4,30.1]
f_bins=[0.9,8.0,809]

In [65]:
rfm_df['r_score']=pd.cut(rfm_df['diff'],r_bins,labels=[i for i in range(len(r_bins)-1,0,-1)])
rfm_df['f_score']=pd.cut(rfm_df['item_id'],f_bins,labels=[i+1 for i in range(len(f_bins)-1)])
rfm_df['rfm_group']=rfm_df['r_score'].astype('str')+rfm_df['f_score'].astype('str')

In [66]:
rfm_df.head()

,user_id,diff,item_id,r_score,f_score,rfm_group
0,4913,2,6,2,1,21
1,6118,1,1,2,1,21
2,7528,5,6,1,1,11
3,7591,5,21,1,2,12
4,12645,4,8,2,1,21


In [67]:
rfm_table=rfm_df.pivot_table(index='r_score',columns='f_score',values='diff',aggfunc='count')

In [68]:
rfm_table

f_score,1,2
r_score,,
2,1721,3179
1,2767,1219


从分析结果看绝大多数用户属于重要价值用户和重要保留用户，我们对不同群体的用户需要制定不同的运营策略。重要价值用户是最优质的用户群体，应重点关注，既要保持其粘性，又要继续引导消费，可为这类用户提供vip服务；重要发展用户的特点是近期有消费但频次不高，策略是提高其消费次数，具体措施有促销活动提醒和优惠卷活动等；重要保持用户的特点是消费频次高但有一段时间没有消费，策略是重新唤醒，通过app消息推送，以及站外广告营销吸引其注意力，促进复购；重要挽留用户近期没有消费且频次不高，若不加以挽留，会有流失的可能，对于这类用户一方面需要保持曝光量，持续推送活动和优惠信息，另一方面需要进一步研究其兴趣和需求，才能采取有效的运营策略


## 商品分析
 1. 统计销售量前十的商品和商品种类,以及相应的购买转化率，按商品贡献度进行帕累托分析。

销售量前十的商品和商品种类

In [69]:
data_item=data[data['behavior_type']=='buy'].groupby('item_id')['user_id'].count().reset_index().sort_values(by='user_id',ascending=False)

In [70]:
item_top10=data_item[:10]

In [71]:
item_top10

,item_id,user_id
69477,303205878,50
3140,14087919,35
26432,115124482,31
55529,243091690,29
38344,167074648,28
29397,127914633,24
25080,109259240,24
23324,101795752,23
85739,374214353,23
3849,17065447,22


In [72]:
data_itemcategory=data[data['behavior_type']=='buy'].groupby('item_category')['user_id'].count().reset_index().sort_values(by='user_id',ascending=False)[:10]

In [73]:
data_itemcategory

,item_category,user_id
2055,6344,2208
593,1863,2000
1682,5232,1611
2279,6977,1324
2908,8877,1072
2121,6513,1059
1736,5399,1054
1076,3424,1053
1091,3472,1038
2598,7957,997


In [74]:
data_itemm_top10_detail=pd.merge(item_top10,data,how='inner',on='item_id')
data_itemm_top10_detail

,item_id,user_id_x,user_id_y,behavior_type,item_category,time,date,hour
0,303205878,50,91256329,buy,13500,2014-12-14 23:00:00,2014-12-14,23
1,303205878,50,110666558,pv,13500,2014-11-26 13:00:00,2014-11-26,13
2,303205878,50,35644209,pv,13500,2014-11-18 20:00:00,2014-11-18,20
3,303205878,50,35644209,pv,13500,2014-11-19 23:00:00,2014-11-19,23
4,303205878,50,35644209,pv,13500,2014-11-21 18:00:00,2014-11-21,18
...,...,...,...,...,...,...,...,...
2791,17065447,22,85336332,buy,8877,2014-12-06 19:00:00,2014-12-06,19
2792,17065447,22,85336332,buy,8877,2014-11-27 12:00:00,2014-11-27,12
2793,17065447,22,85336332,buy,8877,2014-11-27 12:00:00,2014-11-27,12
2794,17065447,22,85336332,buy,8877,2014-12-06 19:00:00,2014-12-06,19


In [75]:
# data_itemm_top10_detail=data_itemm_top10_detail.groupby('item_id')['behavior_type'].value_counts().unstack()
# data_itemm_top10_detail_unstack

In [76]:
data_itemm_top10_detail_1=data_itemm_top10_detail.groupby(['item_id','behavior_type'])['user_id_y'].nunique()
data_itemm_top10_detail_1=data_itemm_top10_detail_1.unstack().fillna(0)

In [77]:
data_itemm_top10_detail_1['ratio']=data_itemm_top10_detail_1['buy']/data_itemm_top10_detail_1['pv']*100
data_itemm_top10_detail_1=data_itemm_top10_detail_1.reset_index()
data_itemm_top10_detail_1

behavior_type,item_id,buy,cart,fav,pv,ratio
0,14087919,25.0,29.0,12.0,205.0,12.195122
1,17065447,1.0,1.0,0.0,2.0,50.000000
2,101795752,2.0,2.0,0.0,6.0,33.333333
3,109259240,2.0,2.0,0.0,13.0,15.384615
4,115124482,1.0,0.0,0.0,2.0,50.000000
5,127914633,1.0,1.0,0.0,2.0,50.000000
6,167074648,28.0,17.0,3.0,111.0,25.225225
7,243091690,3.0,3.0,2.0,20.0,15.000000
8,303205878,20.0,36.0,8.0,162.0,12.345679
9,374214353,14.0,21.0,2.0,85.0,16.470588


In [78]:
#把ratio的折线图再加入
c = (
    Bar()
    .add_xaxis(
   data_itemm_top10_detail_1['item_id'].astype(str).tolist()
    )
  .add_yaxis("pv", data_itemm_top10_detail_1['pv'].tolist())
    .add_yaxis("cart", data_itemm_top10_detail_1['cart'].tolist())
      .add_yaxis("fav", data_itemm_top10_detail_1['fav'].tolist())
      .add_yaxis("buy", data_itemm_top10_detail_1['buy'].tolist())
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-15)),
        title_opts=opts.TitleOpts(title="销售前10的商品", subtitle="各个行为汇总"),
    )
    
)
c.render_notebook()

商品帕累托分析

In [79]:
item_table=data.pivot_table(index='item_category',columns='behavior_type',aggfunc='size').fillna(0).sort_values(by='buy',ascending=False)
item_table['购买累加']=item_table['buy'].cumsum()
item_table['占比']=item_table['购买累加']/item_table['buy'].sum()*100
item_table['type']=item_table['占比'].apply(lambda x:1 if x<80 else 2)

In [80]:
item_table

behavior_type,buy,cart,fav,pv,购买累加,占比,type
item_category,,,,,,,
6344,2208.0,3822.0,1660.0,85369.0,2208.0,1.836862,1
1863,2000.0,9309.0,10200.0,371738.0,4208.0,3.500686,1
5232,1611.0,4486.0,2597.0,135506.0,5819.0,4.840897,1
6977,1324.0,2007.0,273.0,22806.0,7143.0,5.942348,1
8877,1072.0,1974.0,1247.0,63396.0,8215.0,6.834158,1
...,...,...,...,...,...,...,...
6299,0.0,0.0,0.0,25.0,120205.0,100.000000,2
6297,0.0,0.0,0.0,1.0,120205.0,100.000000,2
6291,0.0,0.0,0.0,4.0,120205.0,100.000000,2


In [81]:
item_table.groupby('type')['占比'].count()/item_table['buy'].count()*100

type
1     8.142665
2    91.857335
Name: 占比, dtype: float64

由帕累托分析可得，8.14%的商品类别的销量占了总销量的80%，比较贴近二八定律。

 2. 商品推荐

用tf-id计算每个用户每个商品的权重
每个用户的每个商品的个数/每个用户所购买的所有商品个数*lg(所有商品的个数/该商品的个数)

In [82]:
data_r=data.groupby(['user_id','item_id'])['time'].count().reset_index().rename(columns={'time':'time_count'})
data_r

,user_id,item_id,time_count
0,4913,315532,1
1,4913,876969,6
2,4913,2741340,1
3,4913,3007091,3
4,4913,4210607,1
...,...,...,...
4686899,142455899,402191809,2
4686900,142455899,402919430,5
4686901,142455899,402935883,3
4686902,142455899,403718848,2


In [83]:
data_user_count=data.groupby(['user_id'])['time'].count().reset_index().rename(columns={'time':'user_item_sum'})


In [84]:
df_tfidf=pd.merge(data_r,data_user_count,on='user_id',how='left')
df_tfidf.head()

,user_id,item_id,time_count,user_item_sum
0,4913,315532,1,1742
1,4913,876969,6,1742
2,4913,2741340,1,1742
3,4913,3007091,3,1742
4,4913,4210607,1,1742


每个商品的行为数

In [85]:
df_item_count=df_tfidf.groupby('item_id')['time_count'].sum().reset_index().rename(columns={'time_count':'item_sum'})
df_item_count.head()

,item_id,item_sum
0,64,2
1,496,2
2,528,2
3,579,2
4,581,4


所有商品的行为数总和

In [86]:
df_item_count['sum']=df_item_count.item_sum.sum()

In [87]:
df_tfidf_whole=pd.merge(df_tfidf,df_item_count,on='item_id',how='left')
df_tfidf_whole.head()

,user_id,item_id,time_count,user_item_sum,item_sum,sum
0,4913,315532,1,1742,47,12256906
1,4913,876969,6,1742,12,12256906
2,4913,2741340,1,1742,4,12256906
3,4913,3007091,3,1742,3,12256906
4,4913,4210607,1,1742,4,12256906


In [88]:
df_tfidf_whole['tf_idf']=df_tfidf_whole.time_count/df_tfidf_whole.user_item_sum*(np.log10(df_tfidf_whole['sum']/df_tfidf_whole.item_sum))

In [89]:
df_tfidf_whole.head()

,user_id,item_id,time_count,user_item_sum,item_sum,sum,tf_idf
0,4913,315532,1,1742,47,12256906,0.003109
1,4913,876969,6,1742,12,12256906,0.020698
2,4913,2741340,1,1742,4,12256906,0.003723
3,4913,3007091,3,1742,3,12256906,0.011386
4,4913,4210607,1,1742,4,12256906,0.003723


In [90]:
data5=data.groupby(['user_id','item_id','behavior_type','item_category','date'],as_index=False)['time'].count()
data5.head()

,user_id,item_id,behavior_type,item_category,date,time
0,4913,315532,pv,3431,2014-12-16,1
1,4913,876969,pv,4582,2014-11-27,5
2,4913,876969,pv,4582,2014-11-30,1
3,4913,2741340,pv,1308,2014-11-30,1
4,4913,3007091,pv,7737,2014-11-27,3


In [91]:
data_tfidf=pd.merge(data5,df_tfidf_whole[['user_id','item_id','tf_idf']],on=['user_id','item_id'],how='left')
data_tfidf.head()
data_tfidf.behavior_type.value_counts()

pv      5266185
cart     329275
fav      239469
buy      106119
Name: behavior_type, dtype: int64

In [92]:
data_tfidf['weight']=0.3
data_tfidf.loc[data_tfidf['behavior_type']=='fav','weight']=0.5
data_tfidf.loc[data_tfidf['behavior_type']=='cart','weight']=1
data_tfidf.loc[data_tfidf['behavior_type']=='buy','weight']=1.5

In [93]:
data_tfidf.date.max()

Timestamp('2014-12-18 00:00:00')

In [94]:
def time_weight(x):
    date_time_now=datetime.strptime('2014-12-19','%Y-%m-%d')-x
    date_diff=date_time_now.days
    time_ratio=np.exp(date_diff*(-0.1556))
    return time_ratio

In [95]:
data_tfidf['time_ratio']=data_tfidf['date'].apply(lambda x:time_weight(x))

In [96]:
data_tfidf.head()

,user_id,item_id,behavior_type,item_category,date,time,tf_idf,weight,time_ratio
0,4913,315532,pv,3431,2014-12-16,1,0.003109,0.3,0.627005
1,4913,876969,pv,4582,2014-11-27,5,0.020698,0.3,0.032608
2,4913,876969,pv,4582,2014-11-30,1,0.020698,0.3,0.052006
3,4913,2741340,pv,1308,2014-11-30,1,0.003723,0.3,0.052006
4,4913,3007091,pv,7737,2014-11-27,3,0.011386,0.3,0.032608


In [97]:
data_tfidf['act_weight']=data_tfidf['weight']*data_tfidf['time_ratio']*data_tfidf['time']*data_tfidf['tf_idf']

In [98]:
user_data=data_tfidf[data_tfidf['behavior_type']=='buy']

In [99]:
user_data.head()

,user_id,item_id,behavior_type,item_category,date,time,tf_idf,weight,time_ratio,act_weight
88,4913,44957435,buy,9127,2014-12-07,2,0.025351,1.5,0.154556,0.011754
209,4913,92674121,buy,11981,2014-12-13,1,0.042880,1.5,0.393136,0.025287
383,4913,172639700,buy,13381,2014-12-11,1,0.023864,1.5,0.287999,0.010309
788,4913,343170253,buy,13381,2014-12-01,1,0.070624,1.5,0.060761,0.006437
842,4913,361346418,buy,5065,2014-12-16,1,0.044744,1.5,0.627005,0.042082


In [100]:
user_item_x=user_data.groupby(['user_id','item_id'])['date'].count().reset_index()

In [101]:
user_item_x=user_item_x[['user_id','item_id']]

In [102]:
user_item_x_y=pd.merge(user_item_x,user_item_x,how='inner',on='user_id')

In [103]:
user_item_x_y

,user_id,item_id_x,item_id_y
0,4913,44957435,44957435
1,4913,44957435,92674121
2,4913,44957435,172639700
3,4913,44957435,343170253
4,4913,44957435,361346418
...,...,...,...
3078269,142455899,392926185,196233827
3078270,142455899,392926185,206946411
3078271,142455899,392926185,241670126
3078272,142455899,392926185,355436122


In [106]:
user_item_x_y.drop(labels=user_item_x_y[user_item_x_y['item_id_x']==user_item_x_y['item_id_y']].index,axis=0,inplace=True)
user_item_x_y.head()

,user_id,item_id_x,item_id_y
1,4913,44957435,92674121
2,4913,44957435,172639700
3,4913,44957435,343170253
4,4913,44957435,361346418
5,4913,92674121,44957435


In [107]:
user_c=user_item_x_y.groupby(['item_id_x','item_id_y'])['user_id'].nunique().reset_index().rename(columns={'user_id':'c_count'})
user_c.tail(100)

,item_id_x,item_id_y,c_count
2972984,404557036,244827350,1
2972985,404557036,245692215,1
2972986,404557036,246239501,1
2972987,404557036,246919947,1
2972988,404557036,249038809,1
...,...,...,...
2973079,404561696,165389695,1
2973080,404561696,208263844,1
2973081,404561696,264989086,1
2973082,404561696,275360619,1


In [108]:
user_x=user_item_x_y.groupby('item_id_x')['user_id'].nunique().reset_index().rename(columns={'user_id':'x_count'})
user_x.head()

,item_id_x,x_count
0,4273,1
1,7855,1
2,10403,1
3,11289,1
4,12926,1


In [109]:
user_y=user_item_x_y.groupby('item_id_y')['user_id'].nunique().reset_index().rename(columns={'user_id':'y_count'})

In [110]:
u1=pd.merge(user_x,user_c,on='item_id_x',how='inner')

In [111]:
u2=pd.merge(u1,user_y,on='item_id_y',how='inner')

In [112]:
u2.head()

,item_id_x,x_count,item_id_y,c_count,y_count
0,4273,1,14095017,1,1
1,19996336,1,14095017,1,1
2,44383002,1,14095017,1,1
3,74647142,1,14095017,1,1
4,74670314,1,14095017,1,1


In [113]:
u2['power']=u2['c_count']/np.sqrt(u2['y_count']*u2['x_count'])

In [114]:
u2.head()

,item_id_x,x_count,item_id_y,c_count,y_count,power
0,4273,1,14095017,1,1,1.0
1,19996336,1,14095017,1,1,1.0
2,44383002,1,14095017,1,1,1.0
3,74647142,1,14095017,1,1,1.0
4,74670314,1,14095017,1,1,1.0


In [115]:
u_5=user_data.groupby(['user_id','item_id'],as_index=False)['act_weight'].sum()
u_5.head()

,user_id,item_id,act_weight
0,4913,44957435,0.011754
1,4913,92674121,0.025287
2,4913,172639700,0.010309
3,4913,343170253,0.006437
4,4913,361346418,0.042082


In [116]:
u3=pd.merge(u2,u_5,how='left',left_on='item_id_x',right_on='item_id')

In [117]:
u3['point']=u3['power']*u3['act_weight']

In [118]:
u3.drop('item_id',axis=1,inplace=True)

In [119]:
recommend_result=u3.sort_values('point',ascending=False).groupby('user_id').head(30)

In [120]:
recommend_result.head(40)

,item_id_x,x_count,item_id_y,c_count,y_count,power,user_id,act_weight,point
3127659,115124482,1,57292387,1,1,1.000000,23049176,12.402377,12.402377
3127828,115124482,1,170835619,1,1,1.000000,23049176,12.402377,12.402377
3127996,115124482,1,246992706,1,1,1.000000,23049176,12.402377,12.402377
3128024,115124482,1,253077388,1,1,1.000000,23049176,12.402377,12.402377
3127968,115124482,1,237847446,1,1,1.000000,23049176,12.402377,12.402377
3127800,115124482,1,157102194,1,1,1.000000,23049176,12.402377,12.402377
3128052,115124482,1,323047534,1,1,1.000000,23049176,12.402377,12.402377
3127940,115124482,1,208685358,1,1,1.000000,23049176,12.402377,12.402377
3127687,115124482,1,57465374,1,1,1.000000,23049176,12.402377,12.402377
3128080,115124482,1,327762566,1,1,1.000000,23049176,12.402377,12.402377


# 重要分析结论及建议：
1.	整体来讲，数据集中大部分是付费用户，付费转化率很高，复购率也达到了81%，说明该电商平台已经深入我们的生活，我们需要重点培养用户的忠诚度，让用户继续在平台上消费。
2.	在转化率方面，需要提高点击-收藏加购这一环节（流失率最高）。提出两个假设原因，其一是平台商品众多，但大部分销量偏低，无法吸引客户；其二是平台推送的商品与用户喜好不符。经过进一步分析，前者都得到了证实。
3.	在日维度下，浏览量（PV）、访客数（UV）和每客浏览量（PV/UV）总体波动趋势大体相同，从8号开始逐步攀升，并在12号当天暴增到达顶峰，说明双十二活动效果明显，吸引力大。在小时维度下，19时之后逐步上升，在21时达到峰值后逐步下降，19-21时应该是策划运营的重点时间段。
4.	关于用户分组后的运营，对不同的用户群体采用不同的运营策略，例如对于重要保持用户的特点是消费频次高但有一段时间没有消费，策略是重新唤醒，通过app消息推送，以及站外广告营销吸引其注意力，促进复购。
5.	关于验证提出的商品推荐，可以先尝试做A/B测试，找到最合适的推荐方式，不断提升转化率。